In [3]:
from datasets import load_dataset

c:\Users\karim\python_workspace\pytorch_learning\nlp-transformers\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
emotions = load_dataset('emotion')

In [5]:
from transformers import AutoTokenizer, AutoModel


In [6]:
model_ckpt = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = AutoModel.from_pretrained(model_ckpt)

In [7]:
text = "this is a test"
inputs = tokenizer(text, return_tensors="pt")
inputs

{'input_ids': tensor([[ 101, 2023, 2003, 1037, 3231,  102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1]])}

In [8]:
import torch

In [9]:
with torch.no_grad():
    outputs =  model(**inputs)
print(outputs)

BaseModelOutput(last_hidden_state=tensor([[[-0.1565, -0.1862,  0.0528,  ..., -0.1188,  0.0662,  0.5470],
         [-0.3575, -0.6484, -0.0618,  ..., -0.3040,  0.3508,  0.5221],
         [-0.2772, -0.4459,  0.1818,  ..., -0.0948, -0.0076,  0.9958],
         [-0.2841, -0.3917,  0.3753,  ..., -0.2151, -0.1173,  1.0526],
         [ 0.2661, -0.5094, -0.3180,  ..., -0.4203,  0.0144, -0.2149],
         [ 0.9441,  0.0112, -0.4714,  ...,  0.1439, -0.7288, -0.1619]]]), hidden_states=None, attentions=None)


In [10]:
print(outputs.last_hidden_state.size())

torch.Size([1, 6, 768])


In [11]:
hidden_states = outputs.last_hidden_state

In [12]:
hidden_states

tensor([[[-0.1565, -0.1862,  0.0528,  ..., -0.1188,  0.0662,  0.5470],
         [-0.3575, -0.6484, -0.0618,  ..., -0.3040,  0.3508,  0.5221],
         [-0.2772, -0.4459,  0.1818,  ..., -0.0948, -0.0076,  0.9958],
         [-0.2841, -0.3917,  0.3753,  ..., -0.2151, -0.1173,  1.0526],
         [ 0.2661, -0.5094, -0.3180,  ..., -0.4203,  0.0144, -0.2149],
         [ 0.9441,  0.0112, -0.4714,  ...,  0.1439, -0.7288, -0.1619]]])

In [13]:
hidden_states.size()

torch.Size([1, 6, 768])

In [14]:
# getting the cls token 
hidden_states[:, 0, :].size()

torch.Size([1, 768])

In [15]:
# my task is to take a batch
# get the hidden state for the CLS token

# so how could we do it

# first take in the batc so im expecting it to be smth like 
# [{input_id:tensor , input_mask:tensor},{input_id:tensor , input_mask:tensor}, ...... , {input_id:tensor , input_mask:tensor}]

###
# func extract_hidden_state(batch)
# 

###

In [16]:
batch = [
    "this is sentence one",
    "this is senntence two, lets make it uneven", 
    "this is the last sentence"
]
inputs = tokenizer(batch , padding = True, truncation = True, return_tensors = "pt")
inputs['input_ids'].size()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# shape is {input_ids: [[], [], .... []]}
# so if input_ids is a tensor then its shape will be 
# assume n = batch_size, m = len_of_tokens_in_longest_input, and 
# so the shape of the tensor is (n , m) for both the input_ids and the input_mask

In [17]:
# im expecting a lsit of strings to be the batch

def tokenize_input(batch , key):
    return tokenizer(batch[key], padding = True , truncation= True , return_tensors="pt")
def tokenize_input_test(string_list):
    return tokenizer(batch , padding = True , truncation = True , return_tensors = "pt")

In [18]:
def get_hidden_state(batch):
    inputs = { k : v.to(device) for k , v in batch.items() if k in tokenizer.model_input_names }
    print(inputs)
    with torch.no_grad():
        output = model(**inputs)
        last_hidden_state = output.last_hidden_state
    
    return last_hidden_state[: , 0, :]

In [19]:
emotions_train = emotions['train']

In [20]:
emotions_train

Dataset({
    features: ['text', 'label'],
    num_rows: 16000
})

In [29]:
string_list = [
    "this is input one ",
    "this is input two this sentence is longer"
    "this is input three this sentence is much longer"
    "this is input four this sentence is shorter"
]
tokenized_input = tokenize_input_test(string_list)
tokenized_input['input_ids']

tensor([[  101,  2023,  2003,  6251,  2028,   102,     0,     0,     0,     0,
             0,     0,     0],
        [  101,  2023,  2003, 12411, 10111,  5897,  2048,  1010, 11082,  2191,
          2009, 17837,   102],
        [  101,  2023,  2003,  1996,  2197,  6251,   102,     0,     0,     0,
             0,     0,     0]])